# Разработка интерфейсов визуализации данных на Python с помощью Dash (черновик)

---
Данная публикация представляет собой перевод публикации Дилана Кастильо [Develop Data Visualization Interfaces in Python With Dash](https://realpython.com/python-dash/).

---

Раньше создание аналитических веб-приложений требовало знания нескольких языков программирования. Сегодня вы можете создавать интерфейсы визуализации данных на чистом Python. Одним из популярных инструментов для этого является [Dash](https://dash.plotly.com/introduction).

Dash позволяет специалистам по обработке данных демонстрировать свои результаты в интерактивных веб-приложениях. И не нужно быть экспертом в веб-разработке ― создать и развернуть Dash-приложение займет всего пару часов.

В этом руководстве мы рассмотрим:
- Как создать приложение Dash
- Основные компоненты Dash и компоненты HTML
- Настройка стиля приложения
- Использование обратных вызовов для создания интерактивных приложений
- Как развернуть приложение на Heroku


# Что такое Dash?
Dash ― это платформа с открытым исходным кодом для создания интерфейсов визуализации данных. Она была выпущена в 2017 году как библиотека Python, но впоследствии расширена для R и Julia. Dash помогает специалистам по обработке данных создавать аналитические веб-приложения, не требуя передовых знаний в области веб-разработки.

В основе Dash лежат три технологии:
- **Flask** предоставляет функциональность веб-сервера.
- **React.js** отображает веб-интерфейс.
- **Plotly.js** генерирует диаграммы.

Не нужно беспокоиться о совместной работе этих технологий ― это сделает Dash. Нам нужно только написать код на Python, R или Julia и добавить немного CSS.

Dash создан и поддерживается канадской компанией [Plotly](https://plotly.com/). Возможно, вы знаете об этой компании по популярным графическим библиотекам, носящим ее название. Plotly открыла исходный код Dash и выпустила его по лицензии MIT, так что Dash можно использовать бесплатно.

Если вы привыкли анализировать данные или строить визуализации данных с помощью Python, Dash станет полезным дополнением к вашему набору инструментов. Несколько практических примеров того, что можно сделать с помощью Dash:
- [Панель инструментов](https://dash-gallery.plotly.host/dash-web-trader/) для анализа торговых позиций в режиме реального времени
- [Визуализация](https://dash-gallery.plotly.host/dash-uber-rides-demo/) миллионов поездок Uber
- Интерактивный [финансовый отчет](https://dash-gallery.plotly.host/dash-financial-report/)

Другие интересные варианты использования в [галерее приложений Dash](https://dash-gallery.plotly.host/Portal/).


# Начинаем работу с Dash на Python

В этом руководстве мы шаг за шагом создадим информационную панель для [набора данных Kaggle](https://www.kaggle.com/neuromusic/avocado-prices) о продажах и ценах на авокадо в США за период с 2015 по 2018 год.


## Настройка виртуальног окружения

Для разработки приложения нам понадобится новый каталог для хранения кода и данных, а также чистая виртуальная среда Python 3. Чтобы их создать, следуйте инструкциям для вашей операционной системы.

**Windows**. Откройте командную строку и выполните следующие команды:

```Bash
mkdir avocado_analytics && cd avocado_analytics
python -m venv venv
venv\Scripts\activate.bat
```

Первая команда создает каталог проекта и меняет текущую рабочую директорию на. Вторая команда создает виртуальную среду, а последняя команда её активирует. Вместо команды `python` вам может потребоваться указать путь к файлу `python.exe`.

**macOS или Linux**. Выполните  в терминале следующие действия. Их смысл идентичен командам для Windows.

```Bash
mkdir avocado_analytics && cd avocado_analytics
python3 -m venv venv
source venv/bin/activate
```

Далее нам необходимо установить в виртуальное окружение необходимые библиотеки:

```bash
python -m pip install dash==1.13.3 pandas==1.0.5
```

Эта команда установит Dash и pandas в вашу виртуальную среду. Виртуальная среда позволяет использовать определенные версии библиотек, аналогичные тем, что использовались в этом руководстве.

Наконец, вам понадобятся некоторые данные, которые можно загрузить в сопроводительных материалах урока.

Сохраните файл с данными `avocado.csv` в корневом каталоге проекта. К настоящему моменту у вас должна быть виртуальная среда с необходимыми библиотеками и данными в корневой папке вашего проекта. Структура проекта должна выглядеть так:

```
avocado_analytics/
|
├── venv/
|
└── avocado.csv
```


# Как с помощью Dash создать приложение 
Разобьем процессе создания приложения Dash на два этапа:

1. Инициализируем приложение и определим внешний вид с помощью макета приложения (`layout`).
2. Определим посредством обратных вызовов (`callbacks`),  какие части приложения являются интерактивными и на что они реагируют.


## Инициализируем Dash-приложение
Создадим пустой файл `app.py`. Далее мы будем шаг за шагом его заполнять и пояснять происходящее в коде, а в конце раздела вы найдете его содержимое целиком.

Вот несколько первых строк `app.py`:

In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd

data = pd.read_csv("avocado.csv")
data = data.query("type == 'conventional' and region == 'Albany'")
data["Date"] = pd.to_datetime(data["Date"], format="%Y-%m-%d")
data.sort_values("Date", inplace=True)

app = dash.Dash(__name__)

В строках с 1 по 4 мы импортируем необходимые библиотеки: `dash`, `dash_core_components`, `dash_html_components` и `pandas`:

- `dash` поможет инициализировать приложение
- `dash_core_components` позволяет создавать интерактивные компоненты, такие как графики, раскрывающиеся списки или диапазоны дат.
- `dash_html_components` позволяет получить доступ к тегам HTML.
- `pandas` помогает читать и выводить данные в организованной форме.

Далее в строках с 6 по 9 мы считываем данные и предобрабатываем их для использования в панели управления. Мы также фильтруем некоторые данные, поскольку текущая версия информационной панели не является интерактивной, в противном случае некоторые значения на графике не имели бы смысла.

В последней строке мы создаем экземпляр класса Dash. Если вы раньше использовали Flask, то инициализация класса Dash вам знакома. Во Flask мы обычно инициализируем WSGI-приложение  с помощью `Flask(__name__)`. Точно так же для приложений Dash мы используем `Dash(__name__)`.


## Определение макета приложения Dash

Теперь мы определим макет приложения, его внешний вид. В нашем случае макет будет представлять заголовок, описание и два графика.

In [2]:
app.layout = html.Div(
    children=[
        html.H1(children="Avocado Analytics",),
        html.P(
            children="Analyze the behavior of avocado prices"
            " and the number of avocados sold in the US"
            " between 2015 and 2018",
        ),
        dcc.Graph(
            figure={
                "data": [
                    {
                        "x": data["Date"],
                        "y": data["AveragePrice"],
                        "type": "lines",
                    },
                ],
                "layout": {"title": "Average Price of Avocados"},
            },
        ),
        dcc.Graph(
            figure={
                "data": [
                    {
                        "x": data["Date"],
                        "y": data["Total Volume"],
                        "type": "lines",
                    },
                ],
                "layout": {"title": "Avocados Sold"},
            },
        ),
    ]
)

Этот код определяет свойство `layout` объекта `app`. Внешний вид приложения описывается с помощью древовидной структуры, состоящей из компонентов Dash, поставляемых в виде библиотек Python.

В первых строках можно увидеть на практике HTML-компоненты Dash. Мы начинаем с определения родительского компонента `html.Div`, затем в качестве дочерних элементов добавляем заголовок `html.H1` и абзац `html.P` .

Эти компоненты эквивалентны HTML-тегам `div`, `h1` и `p`. Мы можем использовать аргументы компонентов для изменения атрибутов или содержимого тегов. Например, чтобы указать, что находится внутри тега `div`, мы используем в `html.Div` аргумент `children`.

В компонентах есть и другие аргументы, такие как `style`, `className` или `id`, которые относятся к атрибутам тегов HTML. В следующем разделе мы увидим, как использовать некоторые из этих свойств для стилизации панели инструментов.

Таким образом, часть макета, показанная в первых строках этого кода, будет преобразована в следующий HTML-код:

```HTML
<div>
  <h1>Avocado Analytics</h1>
  <p>
    Analyze the behavior of avocado prices and the number
    of avocados sold in the US between 2015 and 2018
  </p>
  <!-- Rest of the app -->
</div>
```

Этот HTML-код отображается, когда мы открываем приложение в браузере. Он следует той же структуре, что и наш код Python, с тегом `div`, содержащим элементы `h1` и `p`.

В следующих строчках того же блока описаны два компоненты `dcc.Graph` из Dash Core Components. Первый отображает средние цены на авокадо за период исследования, а второй ― количество авокадо, проданных в США за тот же период.

Под капотом Dash использует для создания графиков Plotly.js. Компоненты dcc.Graph ожидают [figure object](https://plotly.com/python/figure-structure/) или словарь Python, содержащий данные графика и `layout`, что мы и передаем в нашем случае.

Остались две строки кода, которые помогут нам запустить приложение:

In [4]:
if __name__ == "__main__":
    app.run_server(debug=True,
                   host = '127.0.0.1')

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


SystemExit: 1

/home/leo/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Эти строки позволяют запускать приложение Dash локально, используя встроенный сервер Flask. Параметр `debug = True` из `app.run_server` разрешает горячую перезагрузку: когда вы вносите изменения в приложение, оно автоматически перезагружается без перезапуска сервера.

Наконец, вот полная версия `app.py`. Вы можете скопировать этот код в пустой `app.py` и проверить результат.

In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd

data = pd.read_csv("avocado.csv")
data = data.query("type == 'conventional' and region == 'Albany'")
data["Date"] = pd.to_datetime(data["Date"], format="%Y-%m-%d")
data.sort_values("Date", inplace=True)

app = dash.Dash(__name__)

app.layout = html.Div(
    children=[
        html.H1(children="Avocado Analytics",),
        html.P(
            children="Analyze the behavior of avocado prices"
            " and the number of avocados sold in the US"
            " between 2015 and 2018",
        ),
        dcc.Graph(
            figure={
                "data": [
                    {
                        "x": data["Date"],
                        "y": data["AveragePrice"],
                        "type": "lines",
                    },
                ],
                "layout": {"title": "Average Price of Avocados"},
            },
        ),
        dcc.Graph(
            figure={
                "data": [
                    {
                        "x": data["Date"],
                        "y": data["Total Volume"],
                        "type": "lines",
                    },
                ],
                "layout": {"title": "Avocados Sold"},
            },
        ),
    ]
)

if __name__ == "__main__":
    app.run_server(debug=True,
                   host = '127.0.0.1')

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


SystemExit: 1

/home/leo/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Пришло время запустить приложение. Откройте терминал в корневом каталоге проекта и в виртуальной среде проекта. Запустите `python app.py`, затем перейдите по адресу `http://localhost:8050`.

Всё работает! Панель управления будет выглядеть примерно так:

<img src="https://files.realpython.com/media/barebones_small.929570811d70.jpg" witdht="500">

Теперь у нас есть рабочая версия, но мы ее еще улучшим.


# Управление оформлением панели

Dash очень гибок в настройке внешнего вида приложения. Вы можете использовать собственные файлы CSS или JavaScript, встраивать изображения и настраивать дополнительные параметры.


## Как применить стиль к компонентам Dash

Вы можете стилизовать компоненты двумя способами:
- Использовать аргумент `style` отдельных компонентов
- Предоставить внешний CSS-файл

Аргумент `style` принимает словарь Python с парами ключ-значение, состоящими из имен свойств CSS и значений, которые мы хотим установить.

---
**Примечание**. При указании свойств CSS в аргументе `style` необходимо использовать синтаксис `mixedCase` вместо слов, разделенных дефисом. Например, чтобы изменить цвет фона элемента, необходимо указывать `backgroundColor`, а не `background-color`.

---

Захотев изменить размер и цвет элемента `H1` в `app.py`, мы можем установить аргумент `style` следующим образом:

```Python
html.H1(
    children="Avocado Analytics",
    style={"fontSize": "48px", "color": "red"},
),
```

В этом случае заголовок будет оформлен красным шрифтом размером в 48 пикселей.

Обратной стороной использования аргумента `style` является то, что с ним все труднее работать по мере роста кодовой базы. Если на панели управления присутствует несколько одинаковых компонентов, большую часть кода придется повторить. Вместо этого можно использовать CSS-файл.

Если вы хотите включить собственные локальные CSS- или JavaScript-файлы, в корневом каталоге проекта необходимо создать папку с именем `assets/` и сохранить в нее необходимые файлы. По умолчанию Dash автоматически обслуживает любой файл, включенный в `assets/`.

Затем вы можете использовать аргументы `className` или `id` компонентов, чтобы настроить с помощью CSS их стили. При преобразовании в HTML-теги эти аргументы соответствуют атрибутам `class` и `id`.

Захотев настроить размер шрифта и цвет текста элемента `H1` в `app.py`, мы можем использовать аргумент `className`:

```Python
html.H1(
    children="Avocado Analytics",
    className="header-title",
),

```

Установка аргумента `className` определяет атрибут класса для элемента H1. Затем в CSS-файле в папке `assets/` мы указываем, как хотим, чтобы он выглядел:

```CSS
.header-title {
  font-size: 48px;
  color: red;
}
```

Мы используем селектор классов для форматирования заголовка в вашем файле CSS. Этот селектор настроит формат заголовка. Вы также можете использовать его с другим элементом, который должен разделять формат, установив className = "header-title".

Затем вы измените стиль своей панели.